In [1]:
%load_ext google.cloud.bigquery

import numpy as np
import pandas as pd
from operator import mul
from functools import reduce
from datetime import datetime

#### Fake dataset 1 - prosty graf

In [2]:
fake1 = pd.DataFrame([
     {'prev':'START', 'next':'A', 'prob':0.3}
    ,{'prev':'START', 'next':'B', 'prob':0.7}
    ,{'prev':'A', 'next':'C', 'prob':0.2}
    ,{'prev':'A', 'next':'D', 'prob':0.8}
    ,{'prev':'B', 'next':'D', 'prob':1.0}
    ,{'prev':'D', 'next':'C', 'prob':0.2}
    ,{'prev':'C', 'next':'A', 'prob':0.3}
    ,{'prev':'C', 'next':'N_CONV', 'prob':0.7}
    ,{'prev':'D', 'next':'N_CONV', 'prob':0.1}
    ,{'prev':'D', 'next':'CONV', 'prob':0.7}
])

print(fake1)

    prev    next  prob
0  START       A   0.3
1  START       B   0.7
2      A       C   0.2
3      A       D   0.8
4      B       D   1.0
5      D       C   0.2
6      C       A   0.3
7      C  N_CONV   0.7
8      D  N_CONV   0.1
9      D    CONV   0.7


### Crawler class
Crawls through a given graph of transitions, producing conversion and non-conversion probabilities. With an optional argument a node (or a list of nodes) can be excluded from the graph.

In [149]:
class Crawler:
    __graph = pd.DataFrame()
    __active_crawlers = []
    __paths = []
    __probs_conv = []
    __probs_n_conv = []
    __current_index = 0
    __verbose = False
    __treshold = 3
    __excluded_nodes = []
    __step_count = 0
    
    def __init__(self, history, probs):
        Crawler.__active_crawlers.append(self)
        self.index = Crawler.__current_index
        Crawler.__current_index += 1
        self.history = history
        self.probs = probs
        Crawler.message('Created crawler{}, active crawlers: {} ({})'.format(self.index, len(Crawler.__active_crawlers), self.history))
    
    def __step(self):
        current_node = self.history[-1]
        exits = Crawler.__graph[Crawler.__graph['prev'] == current_node]['next'].to_list()
        to_remove = []
        for e in exits:
            if (self.history.count(e) > Crawler.__treshold) or (e in Crawler.__excluded_nodes):
                to_remove.append(e)
        exits = list(set(exits) - set(to_remove))
        if len(exits) == 0:
            Crawler.message('Crawler{} finished'.format(self.index))
            Crawler.message('\tCrawler{}\'s path: {} '.format(self.index, self.history))
            Crawler.message('\tCrawler{}\'s probs: {} '.format(self.index, self.probs))
            if current_node == 'CONV':
                Crawler.__paths.append(self.history)
                Crawler.__probs_conv.append(self.probs)
            if current_node == 'N_CONV':
                Crawler.__paths.append(self.history)
                Crawler.__probs_n_conv.append(self.probs)

        else:
            for e in exits:             
                p = Crawler.__graph.loc[(Crawler.__graph['prev'] == current_node) & (Crawler.__graph['next'] == e)]['prob'].to_list()
                new = Crawler(self.history.copy(), self.probs.copy())
                new.history.append(e)
                new.probs += p
                new.__step()
        idx = self.index
        Crawler.__active_crawlers.remove(self)
        Crawler.message('Removed crawler{}, active crawlers: {}'.format(idx, len(Crawler.__active_crawlers)))
    
    @staticmethod
    def message(txt):
        if Crawler.__verbose:
            Crawler.__step_count += 1
            step = Crawler.__step_count
            t = Crawler.get_time()
            print('{}: ({}) {}'.format(step, t, txt))
            
    @staticmethod
    def set_verbose(verbose=True):
        Crawler.__verbose = verbose
        
    
    @staticmethod
    def set_treshold(treshold=3):
        Crawler.__treshold = treshold
    
    @staticmethod
    def start():
        if Crawler.__graph.empty:
            Crawler.message('Before using Crawler you must load a graph! (Crawler.load_graph())')
        else:
            c0 = Crawler(['START'], [])
            c0.__step()
            
    @staticmethod
    def load_graph(graph, excluded_nodes = []):
        Crawler.__graph = graph
        Crawler.__excluded_nodes = excluded_nodes
        
    @staticmethod
    def reset():
        Crawler.__graph = pd.DataFrame()
        Crawler.__active_crawlers = []
        Crawler.__paths = []
        Crawler.__probs_conv = []
        Crawler.__probs_n_conv = []
        Crawler.__current_index = 0
        Crawler.__verbose = False
        Crawler.__treshold = 3
        Crawler.__excluded_nodes = []
        Crawler.__step_count = 0
        
    @staticmethod
    def get_time():
        return(datetime.now().strftime("%H:%M:%S.%f"))
            
    @staticmethod
    def result():
        conv = 0
        n_conv = 0
        for plist in Crawler.__probs_conv:
            conv += reduce(mul, plist)
        for plist in Crawler.__probs_n_conv:
            n_conv += reduce(mul, plist)
        return(conv, n_conv)
    

In [150]:
Crawler.reset()
Crawler.load_graph(fake1, ['D'])
Crawler.set_treshold(3)
#Crawler.set_verbose()
Crawler.start()
print(Crawler.result())

(0, 0.04468027199999999)


### Markov class
Uses the Crawler class to loop through all the nodes in the graph and score them. The conversion score is given by the equation: 

\begin{equation*}
\text{score} = 1 - \left(\frac{\text{conversion probability with the node excluded}}{\text{conversion probability of the full graph}}\right)
\end{equation*}

In [151]:
class Markov:
    def __init__(self, graph, depth=3):
        self.graph = graph
        self.depth = depth
        self.nodes = self.__get_nodes(graph)
        self.prob_conv, self.prob_n_conv = self.__run_crawler(graph)
        n_nodes = len(self.nodes)
        results = {
            'Node': self.nodes,
            'prob_conv': [0] * n_nodes,
            'prob_n_conv': [0] * n_nodes,
            'score_conv': [0] * n_nodes,
            'score_n_conv': [0] * n_nodes
        }
        self.results = pd.DataFrame(results).sort_values(by=['Node'])
    
    def __get_nodes(self, graph):
        prev_set = set(fake1['prev'].to_list())
        next_set = set(fake1['next'].to_list())
        neutral_set = set(['START', 'CONV', 'N_CONV'])
        return(list(prev_set.union(next_set) - neutral_set))
    
    def __run_crawler(self, exclude = []):
        Crawler.reset()
        Crawler.load_graph(self.graph, exclude)
        Crawler.set_treshold(self.depth)
        Crawler.start()
        return(Crawler.result())
    
    def __score(self,node):
        node_prob_conv, node_prob_n_conv = self.__run_crawler(node)
        score_conv = 1 - (node_prob_conv / self.prob_conv)
        score_n_conv = 1 - (node_prob_n_conv / self.prob_n_conv)
        self.results.loc[self.results['Node']==node, 'prob_conv'] = node_prob_conv
        self.results.loc[self.results['Node']==node, 'prob_n_conv'] = node_prob_n_conv
        self.results.loc[self.results['Node']==node, 'score_conv'] = score_conv
        self.results.loc[self.results['Node']==node, 'score_n_conv'] = score_n_conv
    
    def run(self):
        for n in self.nodes:
            Markov.message('Scoring {}'.format(n))
            self.__score(n)
        print('Done.')
        
    @staticmethod
    def get_time():
        return(datetime.now().strftime("%H:%M:%S.%f"))
    
    @staticmethod
    def message(txt):
        t = Markov.get_time()
        print('{}\t{}'.format(t, txt))

In [152]:
m = Markov(fake1, 5)
m.run()
m.results

01:57:04.798244	Scoring D
01:57:04.820085	Scoring A
01:57:04.829364	Scoring C
01:57:04.840637	Scoring B
Done.


,Node,prob_conv,prob_n_conv,score_conv,score_n_conv
1,A,0.490000,0.168000,0.304677,0.431070
3,B,0.188341,0.111659,0.732740,0.621868
2,C,0.658000,0.094000,0.066280,0.681670
0,D,0.000000,0.044681,1.000000,0.848689
